In [1]:
import pandas as pd

In [2]:
meta_outfile = "/data5/deepro/ukbiobank/analysis/lifestyle_factors/data/meta_rarecomb_out_3.csv"

In [3]:
df = pd.read_csv(meta_outfile)

In [4]:
enriched_df = df.loc[df.Case_Adj_Pval_BH<0.05].sort_values("Case_Adj_Pval_BH")

In [5]:
combos = 3
enriched_combos = enriched_df.loc[:, [f"Item_{i}" for i in range(1, combos+1)]].reset_index(drop=True)

In [7]:
enriched_combos["combo_num"] = enriched_combos.index + 1

In [8]:
enriched_combos_melted = pd.melt(enriched_combos, id_vars=["combo_num"], value_vars=[f"Item_{i}" for i in range(1, combos+1)])
enriched_combos_melted["type"] = ["gene" if v.startswith("Input_ENSG") else "lifestyle" for v in enriched_combos_melted.value.values]
enriched_combos_melted_pivot = enriched_combos_melted.sort_values(["type", "value"], ascending=False).groupby("combo_num").agg({"value": lambda x: ",".join(x), "type": lambda x: ",".join(x)})
enriched_combos_melted_pivot_concat = pd.concat((enriched_combos_melted_pivot.value.str.split(",", expand=True), enriched_combos_melted_pivot.type.str.split(",", expand=True)), axis=1)
enriched_combos_melted_pivot_concat.columns = [f"Item_{i}" for i in range(1, combos+1)] + [f"Item_type_{i}" for i in range(1, combos+1)]

In [49]:
enriched_combos_melted_pivot_concat_genes = enriched_combos_melted_pivot_concat.loc[enriched_combos_melted_pivot_concat.Item_type_2=="gene"].reset_index(drop=True).loc[:, [f"Item_{i}" for i in range(1, combos+1)]]
enriched_combos_melted_pivot_concat_lifestyle = enriched_combos_melted_pivot_concat.loc[enriched_combos_melted_pivot_concat.Item_type_2=="lifestyle"].reset_index(drop=True).loc[:, [f"Item_{i}" for i in range(1, combos+1)]]

In [72]:
def save_table(save_df, filename):
    cdf = pd.DataFrame()
    for g, df in save_df.groupby(["Item_1", "Item_2"]):
        cdf = pd.concat((cdf, df), axis=0) 
    cdf = cdf.set_index(["Item_1", "Item_2", "Item_3"])
    cdf.to_csv(filename)
    return

In [73]:
save_table(enriched_combos_melted_pivot_concat_genes, "/data5/deepro/ukbiobank/analysis/lifestyle_factors/data/results/tables/combo3_lf_gene_gene.csv")
save_table(enriched_combos_melted_pivot_concat_lifestyle, "/data5/deepro/ukbiobank/analysis/lifestyle_factors/data/results/tables/combo3_lf_lf_gene.csv")

In [40]:
enriched_combos_melted_pivot_concat.reset_index(drop=True).drop(columns=[c for c in enriched_combos_melted_pivot_concat.columns if "type" in c]).sort_values(["Item_1"]).set_index("Item_1")

,Item_2,Item_3
Item_1,,
Input_diet_catsingle_meat_low,Input_ENSG00000162244,Input_ENSG00000114349
Input_diet_catsingle_meat_low,Input_ENSG00000185070,Input_ENSG00000120693
Input_diet_catsingle_meat_low,Input_ENSG00000243477,Input_ENSG00000162244
Input_diet_catsingle_meat_low,Input_ENSG00000243477,Input_ENSG00000114349
Input_diet_integer_beverage_high,Input_ENSG00000197594,Input_ENSG00000197102
...,...,...
Input_physical_activity_catsingle_sleep_morning_high,Input_ENSG00000271079,Input_ENSG00000213215
Input_physical_activity_integer_duration_high,Input_ENSG00000145217,Input_ENSG00000124541
Input_physical_activity_integer_frequency_low,Input_ENSG00000243244,Input_ENSG00000242441


In [28]:
for g, df in enriched_combos_melted_pivot_concat.groupby("Item_1"):
    print(g)
    print(df)

Input_diet_catsingle_meat_low
                                  Item_1                 Item_2  \
combo_num                                                         
94         Input_diet_catsingle_meat_low  Input_ENSG00000243477   
109        Input_diet_catsingle_meat_low  Input_ENSG00000243477   
120        Input_diet_catsingle_meat_low  Input_ENSG00000162244   
127        Input_diet_catsingle_meat_low  Input_ENSG00000185070   

                          Item_3 Item_type_1 Item_type_2 Item_type_3  
combo_num                                                             
94         Input_ENSG00000162244   lifestyle        gene        gene  
109        Input_ENSG00000114349   lifestyle        gene        gene  
120        Input_ENSG00000114349   lifestyle        gene        gene  
127        Input_ENSG00000120693   lifestyle        gene        gene  
Input_diet_integer_beverage_high
                                     Item_1                 Item_2  \
combo_num                             

In [18]:
enriched_combos_melted_pivot_concat.loc[:, ["Item_1", "Item_2", "Item_3"]].set_index(["Item_1", "Item_2"])

,,Item_3
Item_1,Item_2,
Input_mental_health_catsingle_depression_unenthusiasm_high,Input_mental_health_catsingle_bipolar_high,Input_ENSG00000136371
Input_mental_health_integer_depression_unenthusiasm_high,Input_mental_health_catsingle_bipolar_high,Input_ENSG00000170832
Input_mental_health_catsingle_doctor_visit_high,Input_mental_health_catsingle_depression_unenthusiasm_high,Input_ENSG00000176177
Input_mental_health_integer_depression_unenthusiasm_low,Input_mental_health_catsingle_depression_unenthusiasm_high,Input_ENSG00000127412
Input_mental_health_catsingle_irritable_episodes_high,Input_mental_health_catsingle_bipolar_high,Input_ENSG00000161267
...,...,...
Input_mental_health_integer_neuroticism_low,Input_mental_health_catsingle_emotions_low,Input_ENSG00000163071
Input_mental_health_catsingle_depression_unenthusiasm_high,Input_mental_health_catsingle_bipolar_high,Input_ENSG00000172340
Input_mental_health_catsingle_satisfaction_low,Input_mental_health_catsingle_depression_unenthusiasm_high,Input_ENSG00000127241


In [14]:
for group, df in enriched_combos_melted_pivot_concat.groupby(["Item_1", "Item_2"]):
    print(group)
    print(df) 

('Input_diet_catsingle_meat_low', 'Input_ENSG00000162244')
                                  Item_1                 Item_2  \
combo_num                                                         
120        Input_diet_catsingle_meat_low  Input_ENSG00000162244   

                          Item_3 Item_type_1 Item_type_2 Item_type_3  
combo_num                                                             
120        Input_ENSG00000114349   lifestyle        gene        gene  
('Input_diet_catsingle_meat_low', 'Input_ENSG00000185070')
                                  Item_1                 Item_2  \
combo_num                                                         
127        Input_diet_catsingle_meat_low  Input_ENSG00000185070   

                          Item_3 Item_type_1 Item_type_2 Item_type_3  
combo_num                                                             
127        Input_ENSG00000120693   lifestyle        gene        gene  
('Input_diet_catsingle_meat_low', 'Input_ENSG0000024

In [13]:
enriched_combos_melted_pivot_concat.Item_type_3.unique()

array(['gene'], dtype=object)

In [66]:
enriched_combos_melted_pivot_concat.sort_values(["Item_type_2", "Item_2", "Item_3"], ascending=False)

,Item_1,Item_2,Item_3,Item_type_1,Item_type_2,Item_type_3
combo_num,,,,,,
14,Input_mental_health_integer_depression_unenthu...,Input_mental_health_catsingle_irritable_episod...,Input_ENSG00000196557,lifestyle,lifestyle,gene
107,Input_mental_health_catsingle_satisfaction_low,Input_mental_health_catsingle_irritable_episod...,Input_ENSG00000055609,lifestyle,lifestyle,gene
15,Input_mental_health_integer_neuroticism_low,Input_mental_health_catsingle_emotions_low,Input_ENSG00000197798,lifestyle,lifestyle,gene
155,Input_mental_health_integer_neuroticism_low,Input_mental_health_catsingle_emotions_low,Input_ENSG00000163071,lifestyle,lifestyle,gene
16,Input_mental_health_integer_neuroticism_low,Input_mental_health_catsingle_emotions_low,Input_ENSG00000162302,lifestyle,lifestyle,gene
...,...,...,...,...,...,...
136,Input_mental_health_integer_neuroticism_low,Input_ENSG00000138182,Input_ENSG00000122882,lifestyle,gene,gene
133,Input_diet_integer_fruit_low,Input_ENSG00000136937,Input_ENSG00000136936,lifestyle,gene,gene
98,Input_diet_integer_fruit_high,Input_ENSG00000125378,Input_ENSG00000100504,lifestyle,gene,gene


In [68]:
for i1, i2, i3 in enriched_combos_melted_pivot_concat.sort_values(["Item_type_2", "Item_2", "Item_3"], ascending=False).loc[:, ["Item_1", "Item_2", "Item_3"]].values:
    print(f"{i1}\t{i2}\t{i3}")

Input_mental_health_integer_depression_unenthusiasm_high	Input_mental_health_catsingle_irritable_episodes_high	Input_ENSG00000196557
Input_mental_health_catsingle_satisfaction_low	Input_mental_health_catsingle_irritable_episodes_high	Input_ENSG00000055609
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low	Input_ENSG00000197798
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low	Input_ENSG00000163071
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low	Input_ENSG00000162302
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low	Input_ENSG00000148053
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low	Input_ENSG00000119888
Input_mental_health_catsingle_satisfaction_high	Input_mental_health_catsingle_emotions_low	Input_ENSG00000113262
Input_mental_health_integer_neuroticism_low	Input_mental_health_catsingle_emotions_low

In [8]:
for i1, i2, i3 in df.loc[df.Case_Adj_Pval_BH<0.05, ["Item_1", "Item_2", "Item_3"]].values:
    print(f"{i1}\t{i2}\t{i3}")

Input_ENSG00000136371	Input_mental_health_catsingle_bipolar_high	Input_mental_health_catsingle_depression_unenthusiasm_high
Input_ENSG00000259332	Input_mental_health_catsingle_bipolar_high	Input_mental_health_catsingle_depression_unenthusiasm_high
Input_ENSG00000138759	Input_mental_health_catsingle_depression_unenthusiasm_high	Input_mental_health_catsingle_satisfaction_low
Input_ENSG00000124570	Input_mental_health_catsingle_bipolar_high	Input_mental_health_catsingle_depression_unenthusiasm_high
Input_diet_integer_cereal_low	Input_ENSG00000112852	Input_ENSG00000254221
Input_ENSG00000145242	Input_mental_health_catsingle_bipolar_high	Input_mental_health_catsingle_satisfaction_low
Input_ENSG00000055609	Input_mental_health_catsingle_irritable_episodes_high	Input_mental_health_catsingle_satisfaction_low
Input_ENSG00000099331	Input_mental_health_catsingle_depression_unenthusiasm_high	Input_mental_health_catsingle_irritable_episodes_high
Input_ENSG00000164344	Input_mental_health_catsingle_depr